$s$

In [47]:
import ipywidgets as widgets
out = widgets.Output()
with out:
    
    %bookmark 'model_dir'
    current_dir = %pwd
    while 'models' in current_dir:
        %cd ..
        current_dir = %pwd
    %bookmark 'base_dir'
    %cd source_code
    out.clear_output()
%run base
%run gui_v6
with out:
    %cd ..
    %cd models/fourbar
    out.clear_output()


In [48]:
m=model()
m.show()

## VEHICLE DYNAMICS MODELING AND SIMULATION TOOL

In [49]:
m.model_tree()

In [50]:
m.bodies[2].q0

rbs_ground.x     0.0
rbs_ground.y     0.0
rbs_ground.z     0.0
rbs_ground.e0    1.0
rbs_ground.e1    0.0
rbs_ground.e2    0.0
rbs_ground.e3    0.0
dtype: float64

In [58]:
m.topology.edges(data='joint')

OutMultiEdgeDataView([(rbs_crank, rbs_ground, <constraints.revolute object at 0x0000020FB2172208>), (rbs_crank, rbs_connecting_rod, <constraints.spherical object at 0x0000020FB2340A20>), (rbs_connecting_rod, rbs_slider, <constraints.universal object at 0x0000020FB2447CC0>), (rbs_slider, rbs_ground, <constraints.translational object at 0x0000020FB21B2CC0>)])

In [75]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 12 20:41:13 2018

@author: khale
"""


import numpy as np
import scipy as sc

def jacobian_creator(topology):
    
    edgelist = topology.edges(data='joint')
    nodelist = topology.nodes
    
    n_nodes = len(nodelist)
    n_edges = len(edgelist)
            
    jacobian = np.zeros((n_edges+n_nodes,n_nodes),dtype=np.object)
    jacobian.fill(None)
    
    equations = np.zeros((n_edges+n_nodes,1),dtype=np.object)
    
    vel_rhs = np.zeros((n_edges+n_nodes,1),dtype=np.object)
    acc_rhs = np.zeros((n_edges+n_nodes,1),dtype=np.object)
    
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
        
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        eo    = e[2]
        
        ui = node_index[u]
        vi = node_index[v]
        
        if jacobian[ui+n_edges,ui]==None: jacobian[ui+n_edges,ui] = (u.jac,ui)
        if jacobian[vi+n_edges,vi]==None: jacobian[vi+n_edges,vi] = (v.jac,vi)
                
        jacobian[ei,ui] = (eo.jacobian_i,ui,vi)
        jacobian[ei,vi] = (eo.jacobian_j,vi,ui)
        
        equations[ei,0] = (eo.equations,ui,vi)
        if equations[ui+n_edges,0]==0: equations[ui+n_edges,0]=(u.equations,ui)
        if equations[vi+n_edges,0]==0: equations[vi+n_edges,0]=(v.equations,vi)
        
        
        #vel_rhs[ei,0] = (eo.vel_rhs,ui,vi)
        
        acc_rhs[ei,0] = (eo.acc_rhs,ui,vi)
        
      
    def mapper(i,q):
        try:
            fun,bi,bj = i
            return fun(q[bi],q[bj])
        except ValueError:
            fun,bi = i
            return fun(q[bi])
    
    vectorized = np.vectorize(mapper,otypes=[np.object],excluded='q')
    
    return jacobian, equations, vel_rhs, acc_rhs, jacobian.nonzero(), vectorized


def jacobian_evaluator(jac_blocks,nzi,mapper,q):
    A = jac_blocks.copy()
    A[nzi]=mapper(A[nzi],q=q)
    return sc.sparse.bmat(A)


In [76]:
j,e,v,a,nzi,mapper = jacobian_creator(m.topology)

In [77]:
q0=pd.concat([i.q0 for i in m.topology.nodes])

In [62]:
%timeit jacobian_evaluator(j,nzi,mapper,q0.values.reshape(4,7))

13 ms ± 63.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
e

array([[ (<bound method revolute.equations of <constraints.revolute object at 0x0000020FB2172208>>, 0, 1)],
       [ (<bound method spherical.equations of <constraints.spherical object at 0x0000020FB2340A20>>, 0, 2)],
       [ (<bound method universal.equations of <constraints.universal object at 0x0000020FB2447CC0>>, 2, 3)],
       [ (<bound method translational.equations of <constraints.translational object at 0x0000020FB21B2CC0>>, 3, 1)],
       [(<bound method rigid.equations of rbs_crank>, 0)],
       [(<bound method mount.equations of rbs_ground>, 1)],
       [(<bound method rigid.equations of rbs_connecting_rod>, 2)],
       [(<bound method rigid.equations of rbs_slider>, 3)]], dtype=object)

In [79]:
mapper(e[e.nonzero()],q=q0.values.reshape(4,7))

array([ array([[  3.55271368e-15],
       [ -7.10542736e-15],
       [  1.06581410e-14],
       [  5.55111512e-17],
       [ -1.11022302e-16]]),
       array([[  1.42108547e-14],
       [  1.42108547e-14],
       [ -1.42108547e-14]]),
       array([[ -3.55271368e-15],
       [  6.87549264e-15],
       [ -9.52420783e-17],
       [ -1.52655666e-16]]),
       array([[  9.52420783e-18],
       [  2.29934717e-17],
       [  1.62588398e-16],
       [  1.62588398e-16],
       [ -1.11022302e-16]]),
       array([[ -1.11022302e-16]]),
       array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]]),
       array([[ 0.]]), array([[ -1.11022302e-16]])], dtype=object)

In [65]:
qi = m.joints['jcs_C_trans'].i_body.q0
qj = m.joints['jcs_C_trans'].j_body.q0
m.joints['jcs_C_trans'].equations(qi,qj)

array([[  9.52420783e-18],
       [  2.29934717e-17],
       [  1.62588398e-16],
       [  1.62588398e-16],
       [ -1.11022302e-16]])